The code produces a 1% sample of the original dataset that maintains the same distribution of the 'label' column for each date. The following are the steps followed:

Initialization: It sets up a PySpark session and defines the columns in the data that are considered features (excluding 'date' and 'id').

Outlier Detection:
1. For each unique date in the dataset, it filters the data for that date.
2. For each feature column, it calculates the first and third quartiles (Q1 and Q3) and then determines the Interquartile Range (IQR). Using these, it identifies the lower and upper bounds for outliers.
3. A new column is created for each feature to indicate whether a value is an outlier (1 if yes, 0 if no).
4. A combined 'label' column is derived which is set to 1 if any of the features for a given row is an outlier, otherwise 0.
5. The intermediate outlier columns for individual features are then dropped.

Stratified Sampling:
1. For each date, the code calculates the fraction of rows with each label (0 or 1).
2. It then determines the number of samples required for each label such that the total sample size is 1% of the entire dataset and the samples are evenly spread across different dates, while maintaining the same proportion of each label as in the original data.
3. The data for each date is then sampled based on these calculated fractions.

Aggregation: All the sampled dataframes for each date are unioned together to form a final sampled dataframe.
Summary Statistics: The code calculates and provides summary statistics for both the entire dataset (population) and the sampled dataset.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, count, approxQuantile

# Initialize Spark Session
spark = SparkSession.builder.appName("Outlier_labelling").getOrCreate()

# Assuming df is your dataframe
# df = spark.read.... # Read your dataframe here

feature_columns = [c for c in df.columns if c not in ['date', 'id']]
all_dates = df.select("date").distinct().rdd.flatMap(lambda x: x).collect()

sampled_dfs = []  # Store sampled dataframes for each date

# Calculate the desired total sample size and the sample size per date
total_count = df.count()
desired_sample_count = int(0.01 * total_count)  # 1% of total records
desired_count_per_date = desired_sample_count // len(all_dates)

for date in all_dates:
    temp_df = df.filter(col("date") == date)

    # Outlier Detection for each feature column
    for column in feature_columns:
        Q1, Q3 = temp_df.approxQuantile(column, [0.25, 0.75], 0.01)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        temp_df = temp_df.withColumn("outlier_{}".format(column),
                                     when((col(column) < lower_bound) | (col(column) > upper_bound), 1).otherwise(0))

    # Label column - if any column is an outlier, label as 1, otherwise 0
    temp_df = temp_df.withColumn("label", sum(col("outlier_{}".format(c)) for c in feature_columns))

    # Drop intermediary outlier columns
    for column in feature_columns:
        temp_df = temp_df.drop("outlier_{}".format(column))

    # Calculate the fractions of each label for the current date
    label_counts = temp_df.groupBy("label").count().rdd.collectAsMap()
    total_for_date = temp_df.count()
    label_fractions = {label: count / total_for_date for label, count in label_counts.items()}

    # Calculate the desired number of samples for each label
    desired_samples_by_label = {label: int(desired_count_per_date * fraction) for label, fraction in label_fractions.items()}

    # Sample the desired number of records for each label
    samples_for_date = [temp_df.filter(col("label") == label).sample(withReplacement=False, fraction=min(1.0, desired_samples_by_label[label]/label_counts[label])) for label in label_counts.keys()]
    
    sampled_temp_df = samples_for_date[0]
    for s_df in samples_for_date[1:]:
        sampled_temp_df = sampled_temp_df.union(s_df)
    
    sampled_dfs.append(sampled_temp_df)

# Union all sampled dataframes
final_sampled_df = sampled_dfs[0]
for s_df in sampled_dfs[1:]:
    final_sampled_df = final_sampled_df.union(s_df)

# Summary Statistics
population_summary = df.describe().toPandas()
sample_summary = final_sampled_df.describe().toPandas()

# Save or display the population and sample summary
# population_summary.show()
# sample_summary.show()

# Rename columns for clarity
population_summary.columns = ['Statistic'] + [f'Population_{col}' for col in population_summary.columns[1:]]
sample_summary.columns = ['Statistic'] + [f'Sample_{col}' for col in sample_summary.columns[1:]]

# Merge the dataframes on 'Statistic' column
comparison_df = population_summary.merge(sample_summary, on='Statistic')

print(comparison_df)

# Stop the Spark Session
spark.stop()
